In [1]:
import re
import numpy as np
import collections
from collections import Counter
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from bokeh.io import output_file, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from sklearn.tree import DecisionTreeRegressor

from sklearn.tree import DecisionTreeClassifier

In [2]:
samp = pd.read_csv('sample_submission.csv')
samp.head()

,Customer Id,Claim
0,H0,1
1,H10000,1
2,H10001,1
3,H10002,1
4,H10003,1


In [3]:
train = pd.read_csv('train_data.csv')
test = pd.read_csv('test_data.csv')

In [4]:
vari = pd.read_csv('VariableDescription.csv')
vari

,Variable,Description
0,Customer Id,Identification number for the Policy holder
1,YearOfObservation,year of observation for the insured policy
2,Insured_Period,duration of insurance policy in Olusola Insura...
3,Residential,is the building a residential building or not
4,Building_Painted,"is the building painted or not (N-Painted, V-N..."
5,Building_Fenced,"is the building fence or not (N-Fenced, V-Not ..."
6,Garden,building has garden or not (V-has garden; O-no...
7,Settlement,Area where the building is located. (R- rural ...
8,Building Dimension,Size of the insured building in m2
9,Building_Type,"The type of building (Type 1, 2, 3, 4)"


In [32]:
vari['Description'].iloc[6]

'building has garden or not (V-has garden; O-no garden)'

In [5]:
train.head()

,Customer Id,YearOfObservation,Insured_Period,Residential,Building_Painted,Building_Fenced,Garden,Settlement,Building Dimension,Building_Type,Date_of_Occupancy,NumberOfWindows,Geo_Code,Claim
0,H14663,2013,1.0,0,N,V,V,U,290.0,1,1960.0,.,1053,0
1,H2037,2015,1.0,0,V,N,O,R,490.0,1,1850.0,4,1053,0
2,H3802,2014,1.0,0,N,V,V,U,595.0,1,1960.0,.,1053,0
3,H3834,2013,1.0,0,V,V,V,U,2840.0,1,1960.0,.,1053,0
4,H5053,2014,1.0,0,V,N,O,R,680.0,1,1800.0,3,1053,0


In [6]:
test.head()

,Customer Id,YearOfObservation,Insured_Period,Residential,Building_Painted,Building_Fenced,Garden,Settlement,Building Dimension,Building_Type,Date_of_Occupancy,NumberOfWindows,Geo_Code
0,H11920,2013,1.000000,0,V,N,O,R,300.0,1,1960.0,3,3310
1,H11921,2016,0.997268,0,V,N,O,R,300.0,1,1960.0,3,3310
2,H9805,2013,0.369863,0,V,V,V,U,790.0,1,1960.0,.,3310
3,H7493,2014,1.000000,0,V,N,O,R,1405.0,1,2004.0,3,3321
4,H7494,2016,1.000000,0,V,N,O,R,1405.0,1,2004.0,3,3321


In [7]:
test['Claim'] = None

In [11]:
test.shape

(3069, 14)

In [70]:
df = pd.concat([train, test])
df.head()

,Customer Id,YearOfObservation,Insured_Period,Residential,Building_Painted,Building_Fenced,Garden,Settlement,Building Dimension,Building_Type,Date_of_Occupancy,NumberOfWindows,Geo_Code,Claim
0,H14663,2013,1.0,0,N,V,V,U,290.0,1,1960.0,.,1053,0
1,H2037,2015,1.0,0,V,N,O,R,490.0,1,1850.0,4,1053,0
2,H3802,2014,1.0,0,N,V,V,U,595.0,1,1960.0,.,1053,0
3,H3834,2013,1.0,0,V,V,V,U,2840.0,1,1960.0,.,1053,0
4,H5053,2014,1.0,0,V,N,O,R,680.0,1,1800.0,3,1053,0


In [67]:
df['Geo_Code'].nunique()

1525

In [55]:
df.isnull().sum()

Customer Id              0
YearOfObservation        0
Insured_Period           0
Residential              0
Building_Painted         0
Building_Fenced          0
Garden                   0
Settlement               0
Building Dimension       0
Building_Type            0
Date_of_Occupancy        0
NumberOfWindows          0
Geo_Code                 0
Claim                 3069
dtype: int64

In [49]:
df.Date_of_Occupancy.unique()

array([1960., 1850., 1800., 1980., 1988., 2013., 2011., 1550., 1900.,
       2007., 1970., 1950., 1700., 1976., 1940., 1920., 1982.,   nan,
       1972., 2010., 1971., 1995., 2006., 1999., 1969., 1985., 1965.,
       1981., 1975., 1997., 2003., 1990., 1930., 1973., 1991., 1974.,
       1978., 1967., 1927., 1952., 1957., 2009., 1977., 1890., 1949.,
       1951., 2008., 1987., 2000., 1962., 1956., 1993., 1912., 1983.,
       1954., 1968., 2004., 1936., 1958., 1966., 1953., 1979., 1946.,
       1955., 2001., 1870., 1961., 2015., 1613., 1964., 1984., 2014.,
       2002., 1986., 1992., 2005., 2012., 1910., 1945., 1864., 1942.,
       1939., 1934., 1998., 1908., 1948., 1860., 1926., 1938., 1895.,
       1545., 1959., 1989., 1996., 1963., 1840., 2016., 1718., 1600.,
       1875., 1898., 1880., 1915., 1994., 1810., 1937., 1907., 1931.,
       1925., 1935., 1824., 1914., 1750., 1846., 1903., 1905., 1906.,
       1830., 1923., 1924., 1928., 1947., 1911., 1901., 1902., 1904.,
       1919., 1896.,

In [29]:
df.Garden.unique()

array(['V', 'O', nan], dtype=object)

In [14]:
df['NumberOfWindows'] = df['NumberOfWindows'].fillna(1)

In [71]:
df['NumberOfWindows'] = df['NumberOfWindows'].apply(lambda x: x.replace('   .', '1'))

In [73]:
df['NumberOfWindows'] = df['NumberOfWindows'].astype(int)

In [72]:
df['NumberOfWindows'] = df['NumberOfWindows'].apply(lambda x: x.replace('>=10', '10'))

In [74]:
df['Building Dimension'] = df['Building Dimension'].fillna(df.groupby(['Building_Type', 'NumberOfWindows'])['Building Dimension'].transform('mean'))

In [75]:
df['Garden'] = df['Garden'].fillna(df['Garden'].mode()[0])

In [76]:
df['Date_of_Occupancy'] = df['Date_of_Occupancy'].fillna(df.groupby(['YearOfObservation'])['Date_of_Occupancy'].transform('mean'))

In [77]:
df['Geo_Code'] = df['Geo_Code'].fillna(df['Geo_Code'].mode()[0])


In [26]:
df.iloc[]

Customer Id           H2037
YearOfObservation      2015
Insured_Period            1
Residential               0
Building_Painted          V
Building_Fenced           N
Garden                    O
Settlement                R
Building Dimension      490
Building_Type             1
Date_of_Occupancy      1850
NumberOfWindows           4
Geo_Code               1053
Claim                     0
Name: 1, dtype: object

In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10229 entries, 0 to 3068
Data columns (total 14 columns):
Customer Id           10229 non-null object
YearOfObservation     10229 non-null int64
Insured_Period        10229 non-null float64
Residential           10229 non-null int64
Building_Painted      10229 non-null object
Building_Fenced       10229 non-null object
Garden                10229 non-null object
Settlement            10229 non-null object
Building Dimension    10229 non-null float64
Building_Type         10229 non-null int64
Date_of_Occupancy     10229 non-null float64
NumberOfWindows       10229 non-null int32
Geo_Code              10229 non-null object
Claim                 7160 non-null object
dtypes: float64(3), int32(1), int64(3), object(7)
memory usage: 1.4+ MB


In [78]:
df = df.drop('Customer Id', axis = 1)

In [79]:
df_new = pd.get_dummies(df.drop('Claim', axis=1), drop_first = True)
df_new.head()

,YearOfObservation,Insured_Period,Residential,Building Dimension,Building_Type,Date_of_Occupancy,NumberOfWindows,Building_Painted_V,Building_Fenced_V,Garden_V,...,Geo_Code_95428,Geo_Code_95488,Geo_Code_95500,Geo_Code_95539,Geo_Code_95555,Geo_Code_95563,Geo_Code_95582,Geo_Code_95585,Geo_Code_95598,Geo_Code_95607
0,2013,1.0,0,290.0,1,1960.0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2015,1.0,0,490.0,1,1850.0,4,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2014,1.0,0,595.0,1,1960.0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
3,2013,1.0,0,2840.0,1,1960.0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,2014,1.0,0,680.0,1,1800.0,3,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [81]:
df_new2= pd.concat([df_new, df['Claim']], axis = 1)

In [82]:
train_c = df_new2[pd.notnull(df_new2['Claim'])]
test_c = df_new2[pd.isnull(df_new2['Claim'])]

In [83]:
X = train_c.drop('Claim', axis = 1)
y = train_c['Claim']

In [87]:
X_train, X_test, y_train, y_test = train_test_split(X, y.astype(int), test_size = 0.2, random_state = 42)

In [88]:
lm = LogisticRegression()

In [89]:
lm.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [90]:
pred = lm.predict(X_test)

In [91]:
lm.score(X_test, y_test)

0.7793296089385475

In [92]:
pred_Test = lm.predict(test_c.drop('Claim', axis = 1))

In [93]:
dat_f = pd.DataFrame({'Customer Id': test['Customer Id'],'Claim':pred_Test.astype(int) })
dat_f.head()

,Customer Id,Claim
0,H11920,0
1,H11921,0
2,H9805,0
3,H7493,0
4,H7494,0


In [94]:
dat_f.to_csv('lm1.csv', index=False)

In [128]:
rf = RandomForestClassifier(random_state = 42,  n_jobs = -1, n_estimators=100)

In [129]:
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [130]:
pred_rf = rf.predict(X_test)

In [131]:
rf.score(X_test, y_test)

0.7674581005586593

In [104]:
pred_r = rf.predict(test_c.drop('Claim', axis = 1))

In [105]:
dat_r = pd.DataFrame({'Customer Id': test['Customer Id'],'Claim':pred_r.astype(int) })
dat_r.head()

,Customer Id,Claim
0,H11920,0
1,H11921,0
2,H9805,0
3,H7493,0
4,H7494,0


In [106]:
dat_r.to_csv('rf1.csv', index=False)

In [168]:
sg = SGDClassifier(alpha = 1000,epsilon=100, random_state=42)

In [169]:
sg.fit(X_train, y_train)

SGDClassifier(alpha=1000, average=False, class_weight=None,
              early_stopping=False, epsilon=100, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=42, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [170]:
pred_sg = sg.predict(X_test)

In [171]:
sg.score(X_test, y_test)

0.7688547486033519

In [104]:
pred_r = rf.predict(test_c.drop('Claim', axis = 1))

In [105]:
dat_r = pd.DataFrame({'Customer Id': test['Customer Id'],'Claim':pred_r.astype(int) })
dat_r.head()

,Customer Id,Claim
0,H11920,0
1,H11921,0
2,H9805,0
3,H7493,0
4,H7494,0


In [188]:
knn = KNeighborsClassifier(n_jobs=-1, n_neighbors=1000)

In [189]:
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=-1, n_neighbors=1000, p=2,
                     weights='uniform')

In [190]:
pred_knn = knn.predict(X_test)

In [191]:
knn.score(X_test, y_test)

0.7667597765363129

In [104]:
pred_r = rf.predict(test_c.drop('Claim', axis = 1))

In [105]:
dat_r = pd.DataFrame({'Customer Id': test['Customer Id'],'Claim':pred_r.astype(int) })
dat_r.head()

,Customer Id,Claim
0,H11920,0
1,H11921,0
2,H9805,0
3,H7493,0
4,H7494,0


In [196]:
from sklearn.preprocessing import StandardScaler, Normalizer, RobustScaler
import xgboost as xgb
import lightgbm as lgb

In [192]:
dt = DecisionTreeClassifier()

In [193]:
dt.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [194]:
pred_dt = dt.predict(X_test)

In [195]:
dt.score(X_test, y_test)

0.7171787709497207

In [104]:
pred_dt = dt.predict(test_c.drop('Claim', axis = 1))

In [105]:
dat_dt = pd.DataFrame({'Customer Id': test['Customer Id'],'Claim':pred_dt.astype(int) })
dat_dt.head()

,Customer Id,Claim
0,H11920,0
1,H11921,0
2,H9805,0
3,H7493,0
4,H7494,0


In [201]:
xg = xgb.XGBClassifier(n_estimators=500)

In [202]:
xg.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=500, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [203]:
pred_xg = xg.predict(X_test)

In [204]:
xg.score(X_test, y_test)

0.7814245810055865

In [205]:
pred_xg = xg.predict(test_c.drop('Claim', axis = 1))

In [206]:
dat_xg = pd.DataFrame({'Customer Id': test['Customer Id'],'Claim':pred_xg.astype(int) })
dat_xg.head()

,Customer Id,Claim
0,H11920,0
1,H11921,0
2,H9805,0
3,H7493,0
4,H7494,0


In [207]:
dat_xg.to_csv('xg1.csv', index=False)